<a href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab02/NumpyExcercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Excercises with numpy

In [5]:
%pip install timed-decorator
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 2.7 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.0 MB 1.7 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/11.0 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.0 MB 1.1 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.0 MB 1.4 MB/s eta 0:00:07
   ------- -------------------------------- 2.1/11.0 MB 1.6 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/11.0 MB 1.8 MB/s eta 0:00:05
   ------------ --------------------------- 3.4/11.0 MB 2.0 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/11.0 MB 2.1 MB/s eta 0:00:04
   ----------------- ---------------------- 4.7/11.0 MB 2.3 MB/s eta 0:00:03
   ------------------- -------------------- 5.2/11.0 MB 2.3 MB/s eta 0:00:03
   --------------

In [6]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from timed_decorator.simple_timed import timed
from typing import Tuple

In [7]:
predicted = np.array([
    1,1,1,0,1,0,1,1,0,0
])
actual = np.array([
    1,1,1,1,0,0,1,0,0,0
])

big_size = 500000
big_actual = np.repeat(actual, big_size)
big_predicted = np.repeat(predicted, big_size)


If you are not familiar with the confusion matrix for binary classification, check https://www.geeksforgeeks.org/confusion-matrix-machine-learning/#what-is-a-confusion-matrix

### Exercise 1

Implement a method to retrieve the confusion matrix values using numpy operations. Aim to make your method faster than the sklearn implementation.

In [13]:
@timed(use_seconds=True, show_args=True)
def tp_fp_fn_tn_sklearn(gt: np.ndarray, pred: np.ndarray) -> Tuple[int, ...]:
    tn, fp, fn, tp = confusion_matrix(gt, pred).ravel()
    return tp, fp, fn, tn


@timed(use_seconds=True, show_args=True)
def tp_fp_fn_tn_numpy(gt: np.ndarray, pred: np.ndarray) -> Tuple[int, ...]:
    tp = np.sum((gt == 1) & (pred == 1))
    tn = np.sum((gt == 0) & (pred == 0))
    fp = np.sum((gt == 0) & (pred == 1))
    fn = np.sum((gt == 1) & (pred == 0))
    
    return tp, fp, fn, tn


tp, fp, fn, tn = tp_fp_fn_tn_numpy(actual, predicted)
print(tp, fp, fn, tn)

assert tp_fp_fn_tn_sklearn(actual, predicted) == tp_fp_fn_tn_numpy(actual, predicted)

tp_fp_fn_tn_numpy(ndarray(10,), ndarray(10,)) -> total time: 0.000173100s
4 2 1 3
tp_fp_fn_tn_sklearn(ndarray(10,), ndarray(10,)) -> total time: 0.000520700s
tp_fp_fn_tn_numpy(ndarray(10,), ndarray(10,)) -> total time: 0.000073800s


In [14]:
rez_1 = tp_fp_fn_tn_sklearn(big_actual, big_predicted)
rez_2 = tp_fp_fn_tn_numpy(big_actual, big_predicted)

assert rez_1 == rez_2

tp_fp_fn_tn_sklearn(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.172739300s
tp_fp_fn_tn_numpy(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.030775700s


```
tp_fp_fn_tn_sklearn(ndarray(5000000,), ndarray(5000000,)) -> total time: 1.362611559s
tp_fp_fn_tn_numpy(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.061580794s
```

### Exercise 2

Implement a method to retrieve the calculate the accuracy using numpy operations.

Accuracy = $\frac{TP + TN}{TP + FP + FN + TN}$

In [16]:
@timed(use_seconds=True, show_args=True)
def accuracy_sklearn(gt: np.ndarray, pred: np.ndarray) -> float:
    return accuracy_score(gt, pred)


@timed(use_seconds=True, show_args=True)
def accuracy_numpy(gt: np.ndarray, pred: np.ndarray) -> float:
    correct_predictions = np.sum(gt == pred)
    return correct_predictions / gt.size

assert accuracy_sklearn(actual, predicted) == accuracy_numpy(actual, predicted)

accuracy_sklearn(ndarray(10,), ndarray(10,)) -> total time: 0.000410100s
accuracy_numpy(ndarray(10,), ndarray(10,)) -> total time: 0.000052300s


In [ ]:
rez_1 = accuracy_sklearn(big_actual, big_predicted)
rez_2 = accuracy_numpy(big_actual, big_predicted)

assert np.isclose(rez_1, rez_2)

```
accuracy_sklearn(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.737005607s
accuracy_numpy(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.045633154s
```

## Excercise 3

Implement a method to calculate the F1-Score using numpy operations. Be careful at corner cases (divide by 0).

Precision = $\frac{TP}{TP + FP}$

Recall = $\frac{TP}{TP + FN}$

F1-Score = $2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

In [21]:
@timed(use_seconds=True, show_args=True)
def f1_score_sklearn(gt: np.ndarray, pred: np.ndarray) -> float:
    return f1_score(gt, pred)


@timed(use_seconds=True, show_args=True)
def f1_score_numpy(gt: np.ndarray, pred: np.ndarray) -> float:
    tp = np.sum((gt == 1) & (pred == 1))
    fp = np.sum((gt == 0) & (pred == 1))
    fn = np.sum((gt == 1) & (pred == 0))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if (precision + recall) == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

print(f1_score_sklearn(actual, predicted))
print(f1_score_numpy(actual, predicted))


# assert f1_score_sklearn(actual, predicted) == f1_score_numpy(actual, predicted)

f1_score_sklearn(ndarray(10,), ndarray(10,)) -> total time: 0.000916400s
0.7272727272727273
f1_score_numpy(ndarray(10,), ndarray(10,)) -> total time: 0.000075400s
0.7272727272727272


In [22]:
rez_1 = f1_score_sklearn(big_actual, big_predicted)
rez_2 = f1_score_numpy(big_actual, big_predicted)

assert np.isclose(rez_1, rez_2)

f1_score_sklearn(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.389090800s
f1_score_numpy(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.024554000s


```
f1_score_sklearn(ndarray(5000000,), ndarray(5000000,)) -> total time: 5.300984584s
f1_score_numpy(ndarray(5000000,), ndarray(5000000,)) -> total time: 0.042523483s
```